
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

In [0]:
%run ../Includes/Copy-Datasets

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/customers-json")
display(files)

In [0]:
SELECT * FROM json.`${dataset.bookstore}/customers-json/export_001.json`

In [0]:
SELECT * FROM json.`${dataset.bookstore}/customers-json/export_*.json`

In [0]:
SELECT * FROM json.`${dataset.bookstore}/customers-json`

In [0]:
SELECT count(*) FROM json.`${dataset.bookstore}/customers-json`

In [0]:
SELECT *,
  input_file_name() source_file
FROM json.`${dataset.bookstore}/customers-json`;

In [0]:
SELECT * FROM text.`${dataset.bookstore}/customers-json`

In [0]:
SELECT * FROM binaryFile.`${dataset.bookstore}/customers-json`

In [0]:
SELECT * FROM csv.`${dataset.bookstore}/books-csv`;

In [0]:
CREATE TABLE books_csv
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  header = "true",
  delimiter = ";"
)
LOCATION "${dataset.bookstore}/books-csv";

In [0]:
SELECT * FROM books_csv;

In [0]:
DESCRIBE EXTENDED books_csv;

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
display(files)

In [0]:
%python
(spark.read
      .table("books_csv")
     .write
        .mode("append")
        .format("csv")
        .option('header','true')
        .option('delimeter',';')
        .save(f"{dataset_bookstore}/books-csv"))

In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/books-csv")
display(files)

In [0]:
SELECT COUNT(*) FROM books_csv;

In [0]:
REFRESH TABLE books_csv;

In [0]:
SELECT COUNT(*) FROM books_csv;

In [0]:
CREATE TABLE customers AS
SELECT * FROM json.`${dataset.bookstore}/customers-json`;

DESCRIBE EXTENDED customers;

In [0]:
CREATE TABLE books_unparsed AS
SELECT * FROM csv.`${dataset.bookstore}/books-csv`;

SELECT * FROM books_unparsed;

In [0]:
--- Creating a temp view to pull data from CSV files ---
CREATE TEMP VIEW books_temp_view
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  path = "${dataset.bookstore}/books-csv/export_*.csv",
  header = "true",
  delimiter = ";"
);

--- Creating a table from the information pulled into the temp view from CSV files ---
CREATE TABLE books AS
  SELECT * FROM books_temp_view;

--- Can now query the info from the CSV files directly from our new delta table in the databricks database ---
SELECT * FROM books;

In [0]:
DESCRIBE EXTENDED books;

# read_files() - New Function for File data ingestion:
## Modern Databricks way to accomplish the same data ingestion instead of the older temp table way above

In [0]:
DROP TABLE IF EXISTS books_two;

CREATE TABLE books_two
AS SELECT * FROM read_files(
    '${dataset.bookstore}/books-csv/export_*.csv',
    format => 'csv',
    header => 'true',
    delimiter => ';'
);

In [0]:
SELECT * FROM books_two;